In [ ]:
import sys
sys.path.append('..')

In [ ]:
import tables
import numpy as np
import pandas as pd
from os import path
import matplotlib.pyplot as plt
from gerumo.visualization.explore import plot_telescope_geometry, plot_observation_scatter, plot_array
%matplotlib inline

In [ ]:
hdf5_filepath = "/home/asuka/Desktop/gamma_20deg_0deg_run1000___cta-prod5-paranal_desert-2147m-Paranal-dark_cone10_merged.DL1.h5"

In [ ]:
hdf5_file = tables.open_file(hdf5_filepath, "r")
source = path.basename(hdf5_filepath)
folder = path.dirname(hdf5_filepath)

# Layout

In [ ]:
array_layout = "configuration/instrument/subarray/layout"
hdf5_file.root[array_layout]

In [ ]:
plot_array(hdf5_file, layout=array_layout)

# Geometries

In [ ]:
LST_geom='/configuration/instrument/telescope/camera/geometry_LSTCam'
CHEC_geom='/configuration/instrument/telescope/camera/geometry_CHEC'
Flash_geom='/configuration/instrument/telescope/camera/geometry_FlashCam'
Nectar_geom='/configuration/instrument/telescope/camera/geometry_NectarCam'
cameras=[LST_geom, CHEC_geom, Flash_geom] #, Nectar_geom]
cameras_names=['LST','SSTC','MSTF'] # ,'MSTN']
hdf5_file.root[cameras[0]]

In [ ]:
for name, camera in zip(cameras_names, cameras):
    camera_table=hdf5_file.root[camera]
    pixels = np.array([[p['pix_x'],p['pix_y']] for p in camera_table]).T
    num_pixels=pixels.shape[1]
    plot_telescope_geometry(name, pixels, num_pixels)
    plt.show()

# Images

In [ ]:
camera='/configuration/instrument/telescope/camera'
images= '/dl1/event/telescope/images/tel_001'
hdf5_file.root[images]

In [ ]:
camera_table=hdf5_file.root[LST_geom]
pix_pos = np.array([[p['pix_x'],p['pix_y']] for p in camera_table]).T

In [ ]:
idx = 510

charge=hdf5_file.root[images][idx]['image']
mask=hdf5_file.root[images][idx]['image_mask']
peak_pos=hdf5_file.root[images][idx]['peak_time']
event_id=hdf5_file.root[images][idx]['event_id']


plot_observation_scatter(charge, peak_pos, mask, pix_pos, 'LST', event_id)

# Events

In [ ]:
event_info_table = "simulation/event/subarray/shower"
event_triggers_table = "/dl1/event/subarray/trigger"

In [ ]:
for i, event in enumerate(hdf5_file.root[event_info_table]):
    if i > 2: break
    print(i, *zip(event.table.colnames, event.fetch_all_fields()), sep="\n\t")

In [ ]:
counters = np.zeros(180)
for i, event_triggers in enumerate(hdf5_file.root[event_triggers_table]):
    if i > 2: break
    print(i, *zip(event_triggers.table.colnames, event_triggers.fetch_all_fields()), sep="\n\t")
    counters[event_triggers.fetch_all_fields()[3]] += 1

# Telescopes

In [ ]:
telescope_parameters = "/dl1/event/telescope/parameters/tel_001"  # Hillas parameters
telescope_images = "/dl1/event/telescope/images/tel_{0}"          # Time peaks and charge images

In [ ]:
flag = True

for i, telescope in enumerate(hdf5_file.root[telescope_parameters]): # + "/tel_001"]):
    if i > 3: break
    print(i, *zip(telescope.table.colnames, telescope.fetch_all_fields()), sep="\n\t")
    